In [1]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC



### Read

In [2]:
x_train = pd.read_csv("train.csv")
x_test = pd.read_csv("test.csv")
x_test = x_test.drop(columns=['id'])
x_train = x_train.drop(columns=['id'])
x_test = x_test.drop(x_test.columns[0], axis=1)
x_train = x_train.drop(x_train.columns[0], axis=1)
x_train = x_train.dropna()
x_test = x_test.dropna()

### Data Preprocessing

In [3]:

x_test,y_test = x_test.drop(['satisfaction'],axis=1),x_test['satisfaction']
y_test = y_test.replace('neutral or dissatisfied',0)
y_test = y_test.replace('satisfied',1)
x_train,y_train = x_train.drop(['satisfaction'],axis=1),x_train['satisfaction']
y_train = y_train.replace('neutral or dissatisfied',0)
y_train = y_train.replace('satisfied',1)

C:\Users\LENOVE\AppData\Local\Temp\ipykernel_6556\1586823674.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_test = y_test.replace('satisfied',1)
C:\Users\LENOVE\AppData\Local\Temp\ipykernel_6556\1586823674.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.replace('satisfied',1)


In [4]:
categorical_cols = x_train.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough'
)

x_train_encoded = preprocessor.fit_transform(x_train)
x_test_encoded = preprocessor.transform(x_test)

scaler = StandardScaler()
scaler.fit(x_train_encoded)
x_train_encoded_scaler = scaler.transform(x_train_encoded)
x_test_encoded_scaler = scaler.transform(x_test_encoded)

### Search Parameters For SVM Model

In [ ]:
svm = SVC()
svm_param_grid = [
    {
        'C': [0.001,0.01,0.1, 1, 10],  
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto'] + [0.001, 0.01, 0.1, 1], 
        'degree': [2, 3], 
        'probability': [True]
    }
]

svm_grid_search = GridSearchCV(svm, svm_param_grid, cv=3, scoring='accuracy')


svm_grid_search.fit(x_train_encoded_scaler, y_train)

print("Best params:", svm_grid_search.best_params_)
print("Best estimator:", svm_grid_search.best_estimator_)
print("Best score:", svm_grid_search.best_score_)

### SVM Model

In [ ]:
# svm = SVC(
#         probability=True,
#         kernel='rbf',
#         random_state=42,
#         C=1,
#         gamma='scale',
#         degree=2,
#     )

# # Train
# svm.fit(x_train_encoded_scaler, y_train)

# # Predict
# y_train_pred = svm.predict(x_train_encoded_scaler)
# y_test_pred = svm.predict(x_test_encoded_scaler)

# # Accuracy
# train_accuracy = accuracy_score(y_train, y_train_pred) * 100
# test_accuracy = accuracy_score(y_test, y_test_pred) * 100

# print(f"Training Accuracy: {train_accuracy:.2f}%")
# print(f"Test Accuracy: {test_accuracy:.2f}%")


ValueError: Found input variables with inconsistent numbers of samples: [4143, 103594]

### Confusion Matrix